In [19]:
# import libraries
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.base.model
import statsforecast
import mlforecast
import neuralforecast
import hierarchicalforecast.core

# import functions
from random import random

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [15]:
# create a class that takes data and variables of interest
class Model:
    def __init__(self, df, x, y):

        # take a PANDAS dataframe
        self.df = df

        # take a list of independent variable name strings
        self.x = x

        # take a dependent variable name string
        self.y = y
    
    # instantiate a particular type of model given a string of its name
    def instance(self, type):
        self.type = type
        if self.type == 'OLS':
            self.model = sm.OLS(self.df[[self.y]], sm.add_constant(self.df[self.x]))
    
    # train the model
    def fit(self, alpha = 0.05):
        if self.type in ['OLS']:
            self.model         = self.model.fit()
            self.points        = self.model.nobs
            self.df_resid      = self.model.df_resid
            self.df_model      = self.model.df_model
            self.alpha         = alpha
            self.coef          = self.model.params
            self.coef_std      = self.model.bse
            self.coef_t        = self.model.tvalues
            self.coef_p        = self.model.pvalues
            self.coef_lower    = self.model.conf_int(alpha = alpha)[0]
            self.coef_upper    = self.model.conf_int(alpha = alpha)[1]
            self.r2            = self.model.rsquared
            self.r2_adj        = self.model.rsquared_adj
            self.f_stat        = self.model.fvalue
            self.f_stat_prob   = self.model.f_pvalue
            self.log_like      = self.model.llf
            self.aic           = self.model.aic
            self.bic           = self.model.bic
            omnibus            = sms.omni_normtest(self.model.resid)
            self.omnibus       = omnibus.statistic
            self.omnibus_p     = omnibus.pvalue
            jarque_bera        = sms.jarque_bera(self.model.resid)
            self.skew          = jarque_bera[2]
            self.kurtosis      = jarque_bera[3]
            self.durbin_watson = sms.durbin_watson(self.model.resid)
            self.jarque_bera   = jarque_bera[0]
            self.jarque_bera_p = jarque_bera[1]
            self.condition_no  = self.model.condition_number
    
    # summarize the model
    def summary(self):
        if self.type in ['OLS']:
            return pd.DataFrame([['type',          'model type',                                                       self.type],
                                 ['points',        'data points',                                                      self.points],
                                 ['df_resid',      'degrees of freedom of residuals',                                  self.df_resid],
                                 ['df_model',      'degrees of freedom of model',                                      self.df_model],
                                 ['y',             'dependent variable',                                               self.y],
                                 ['x',             'independent variable(s)',                                          'constant, ' + ', '.join(self.x)],
                                 ['coef',          'coefficients',                                                     ', '.join([str(round(i, 4)) for i in self.coef])],
                                 ['coef_std',      'standard errors',                                                  ', '.join([str(round(i, 4)) for i in self.coef_std])],
                                 ['coef_t',        't statistics',                                                     ', '.join([str(round(i, 4)) for i in self.coef_t])],
                                 ['coef_p',        'p values',                                                         ', '.join([str(round(i, 4)) for i in self.coef_p])],
                                 ['coef_lower',    f'{int(100 * (1 - self.alpha))}% confidence interval lower bounds', ', '.join([str(round(i, 4)) for i in self.coef_lower])],
                                 ['coef_upper',    f'{int(100 * (1 - self.alpha))}% confidence interval upper bounds', ', '.join([str(round(i, 4)) for i in self.coef_upper])],
                                 ['r2',            'R^2',                                                              self.r2],
                                 ['r2_adj',        'adjusted R^2',                                                     self.r2_adj],
                                 ['f_stat',        'F statistic',                                                      self.f_stat],
                                 ['f_stat_prob',   'F statistic p value',                                              self.f_stat_prob],
                                 ['log_like',      'Logarithmic likelihood',                                           self.log_like],
                                 ['aic',           'Akaike information criterion',                                     self.aic],
                                 ['bic',           'Bayesian information criterion',                                   self.bic],
                                 ['omnibus',       'Omnibus',                                                          self.omnibus],
                                 ['omnibus_p',     'Omnibus p value',                                                  self.omnibus_p],
                                 ['skew',          'Skew',                                                             self.skew],
                                 ['kurtosis',      'Kurtosis',                                                         self.kurtosis],
                                 ['durbin_watson', 'Durbin-Watson test',                                               self.durbin_watson],
                                 ['jarque_bera',   'Jarque-Bera test',                                                 self.jarque_bera],
                                 ['jarque_bera_p', 'Jarque-Bera test p value',                                         self.jarque_bera_p],
                                 ['condition_no',  'Condition number',                                                 self.condition_no]],
                       columns = ['attribute',     'title',                                                            'value'])

In [16]:
# test Model class
data = pd.DataFrame({'x1': [random() for i in range(50)],
                     'x2': [random() for i in range(50)],
                     'y':  [random() for i in range(50)]})
model = Model(data,['x1', 'x2'], 'y')
model.instance('OLS')
model.fit()
display(model.summary())

# compare native statsmodels summary
print(model.model.summary())

,attribute,title,value
0,type,model type,OLS
1,points,data points,50.0
2,df_resid,degrees of freedom of residuals,47.0
3,df_model,degrees of freedom of model,2.0
4,y,dependent variable,y
5,x,independent variable(s),"constant, x1, x2"
6,coef,coefficients,"0.5876, 0.1149, -0.1809"
7,coef_std,standard errors,"0.1095, 0.158, 0.1554"
8,coef_t,t statistics,"5.3638, 0.7271, -1.1642"
9,coef_p,p values,"0.0, 0.4707, 0.2502"


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.8401
Date:                Tue, 31 Dec 2024   Prob (F-statistic):              0.438
Time:                        16:00:38   Log-Likelihood:                -9.3566
No. Observations:                  50   AIC:                             24.71
Df Residuals:                      47   BIC:                             30.45
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5876      0.110      5.364      0.0

In [20]:
# create a class that inherits base classes from statsmodels and nixtla

class Base(statsmodels.base.model.Model,
           statsforecast.StatsForecast,
           mlforecast.MLForecast,
           neuralforecast.NeuralForecast,
           hierarchicalforecast.core.HierarchicalReconciliation):
    pass